# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_query`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_query.html)
- Define feature and target variables X and Y

In [16]:
# import libraries

import sys
import re
import nltk

import pandas as pd
import pickle
from sqlalchemy import create_engine

# import statements
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [29]:
def load_data(database_filepath):
    # load data from database
    engine = create_engine('sqlite:///{}'.format(database_filepath))
    df = pd.read_sql_table('messages_categories', engine)
    X = df.message.values
    Y = df.iloc[:, 4:].values
    category_names = (df.columns[4:]).tolist()
    return X, Y, category_names

In [30]:
X, Y, category_names = load_data('../data/DisasterResponse.db')
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [31]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X = df.message.values
Y = df.iloc[:, 4:].values

In [10]:
type(X)

numpy.ndarray

### 2. Write a tokenization function to process your text data

In [43]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Reduce words to their root form
    lemmatized = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    # iterate through each token
    clean_words = []
    for tokens in lemmatized:
        
        # lemmatize, and remove leading/trailing white space
        clean_words.append(tokens.strip())
        
    return clean_words

In [44]:
tokenize(df.message[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [34]:
pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
            ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, y_train)



### 5. Test your model
Report the accuracy, precision and recall on both the training set and the test set. You can use sklearn's `classification_report` function here. 

In [40]:
# predict on test data
y_pred = pipeline.predict(X_test)

print("---- Classification Report/n")
for i in range(len(category_names)):
    print("Label:", category_names[i])

    print(classification_report(y_test[:, i], y_pred[:, i]))

---- Classification Report/n
Label: related
             precision    recall  f1-score   support

          0       0.63      0.49      0.55      1519
          1       0.85      0.91      0.88      4980
          2       0.28      0.38      0.32        55

avg / total       0.80      0.81      0.80      6554

Label: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5440
          1       0.81      0.41      0.55      1114

avg / total       0.88      0.88      0.87      6554

Label: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      1.00      0.99      6554

Label: aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3833
          1       0.75      0.59      0.66      2721

avg / total       0.75      0.75      0.74      65

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# set parameters for grid search
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'tfidf__use_idf': [True, False],
    'tfidf__norm': ['l1', 'l2']
}

# optimize model
model = GridSearchCV(pipeline, param_grid=parameters,
                     cv=2, verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [17]:
classification_report??

In [19]:
with open('../models/classifier.pkl', 'rb') as f:
    model = pickle.load(f)

In [20]:
model.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [21]:
model.best_score_

0.24012969673850848

In [23]:
model.best_params_

{'clf__estimator__criterion': 'entropy',
 'tfidf__norm': 'l1',
 'tfidf__use_idf': False}

In [24]:
model.param_grid

{'clf__estimator__criterion': ['gini', 'entropy'],
 'tfidf__norm': ['l1', 'l2'],
 'tfidf__use_idf': [True, False]}